### Multi Classification 
: 다중분류  
활성화 함수(Activation function) 으로 softmax함수 가 사용된다


In [1]:
import tensorflow as tf
import numpy as np

In [3]:
# 데이터 가져오기
xy =np.loadtxt('data-04-zoo.csv',delimiter=',',dtype=np.float32)
print(xy.shape)  # (101,17)

# 학습 데이터 : 70% , 70개
x_train = xy[__,__]
y_train = xy[__,__]

# 검증 데이터 : 30%, 31개
x_test = xy[__,__]
y_test = xy[__,__]

(101, 17)
